## Benefícios Concedidos do INSS

### Bibliotecas

In [1]:
# Importando as bibliotecas
import pandas as pd
from dateutil.relativedelta import relativedelta
import glob
from pathlib import Path

### Bases de Dados

#### Lendo os arquivos das bases que serão utilizadas

In [ ]:
# Caminho para a pasta contendo os arquivos Excel
caminho_bases = Path('bases_inss')
padrao_arquivos = 'ben_*.xlsx'

colunas_desejadas = ["Competência concessão", "Espécie_NUM", "Espécie_NOME", "CID_NUM_NOME", "Despacho_NOME", "Dt Nascimento", "Sexo.", "Clientela", "Mun Resid", "Vínculo dependentes", "Forma Filiação", "UF", "Qt SM RMI", "Ramo Atividade", "Dt DCB", "Dt DDB", "Dt DIB", "País de Acordo Internacional", "Classificador PA"]

lista_dataframes = []
for arquivo_excel in caminho_bases.glob(padrao_arquivos):
    print(f"Lendo o arquivo: {arquivo_excel.name}")
    try:
        base_compl1 = pd.read_excel(arquivo_excel)

        base_compl2 = base_compl1.reindex(columns=colunas_desejadas)

        lista_dataframes.append(base_compl2)
        print(f"  -> Colunas selecionadas e adicionadas do arquivo: {arquivo_excel.name}")

    except Exception as e:
        print(f"Erro ao processar {arquivo_excel.name}: {e}")

if lista_dataframes:
    base_completa = pd.concat(lista_dataframes, ignore_index=True)
    print("Bases concatenadas com sucesso com colunas selecionadas!")
    print("Colunas na base_completa:", base_completa.columns.tolist())
else:
    print("Nenhum arquivo encontrado ou lido com sucesso.")
    base_completa = pd.DataFrame(columns=colunas_desejadas)

Lendo o arquivo: ben_ago_24.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_ago_24.xlsx
Lendo o arquivo: ben_dez_24.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_dez_24.xlsx
Lendo o arquivo: ben_fev_25.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_fev_25.xlsx
Lendo o arquivo: ben_jan_25.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_jan_25.xlsx
Lendo o arquivo: ben_jul_24.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_jul_24.xlsx
Lendo o arquivo: ben_jun_24.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_jun_24.xlsx
Lendo o arquivo: ben_mar_25.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_mar_25.xlsx
Lendo o arquivo: ben_nov_24.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_nov_24.xlsx
Lendo o arquivo: ben_out_24.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_out_24.xlsx
Lendo o arquivo: ben_set_24.xlsx
  -> Colunas selecionadas e adicionadas do arquivo: ben_se

#### Tratamento dos dados

In [3]:
# Transformando a coluna 'Competência concessão' em data
base_completa["Competência concessão"] = pd.to_datetime(base_completa["Competência concessão"])

In [4]:
# Selecionando apenas as linhas em que a coluna do CID não é zerada ou sem informação
base_inteira = base_completa[(base_completa["CID_NUM_NOME"] != 'Zerados') & (base_completa["CID_NUM_NOME"] != '{ñ class}') & (base_completa["CID_NUM_NOME"] != 'Em Branco')]

In [5]:
# Substituindo os status do beneficiário para facilitar os agrupamentos para futuras análises
base_inteira['Forma Filiação'] = base_inteira['Forma Filiação'].replace({'Empresário': 'Autônomo', 'Equiparado a Autônomo': 'Autônomo', 'Trabalhador Avulso': 'Autônomo'})

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\177539646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_inteira['Forma Filiação'] = base_inteira['Forma Filiação'].replace({'Empresário': 'Autônomo', 'Equiparado a Autônomo': 'Autônomo', 'Trabalhador Avulso': 'Autônomo'})


In [6]:
# Calculando a idade dos beneficiários com base na data de início do benefício e a data de nascimento
base_inteira['Idade1'] = (base_inteira['Dt DIB'] - base_inteira['Dt Nascimento']).dt.days / 365.25

# Arredondando a idade dos beneficiários para números inteiros
base_inteira['Idade'] = base_inteira['Idade1'].round()

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\3588559325.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_inteira['Idade1'] = (base_inteira['Dt DIB'] - base_inteira['Dt Nascimento']).dt.days / 365.25
C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\3588559325.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_inteira['Idade'] = base_inteira['Idade1'].round()


In [7]:
# Excluindo a coluna 'Idade1'
base_inteira = base_inteira.drop(columns=['Idade1'])

In [8]:
# Ajustando as datas zeradas
base_inteira['Dt DCB'] = base_inteira['Dt DCB'].replace('00/00/0000', '01/01/1900')
base_inteira['Dt DDB'] = base_inteira['Dt DDB'].replace('00/00/0000', '01/01/1900')
# Transformando as colunas 'Dt DCB', 'Dt DIB' e 'Dt DDB' em data
base_inteira["Dt DIB"] = pd.to_datetime(base_inteira["Dt DIB"])
base_inteira['Dt DCB'] = pd.to_datetime(base_inteira['Dt DCB'])
base_inteira['Dt DDB'] = pd.to_datetime(base_inteira['Dt DDB'])

In [9]:
# Selecionando apenas as linhas em que as colunas de data não são zeradas ou sem informação
base_inteira1 = base_inteira[(base_inteira["Dt DCB"] != '01/01/1900') & (base_inteira["Dt DDB"] != '01/01/1900') & (base_inteira["Dt DIB"] != '01/01/1900')]

In [10]:
# Calculando o tempo de espera para o despacho do benefício e o tempo do benefício
base_inteira1['TEMPO_ESPERA_DESPACHO_DIAS'] = (base_inteira1['Dt DDB'] - base_inteira1['Dt DIB']).dt.days
base_inteira1['TEMPO_BENEF_DIAS'] = (base_inteira1['Dt DCB'] - base_inteira1['Dt DIB']).dt.days

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\1260596745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_inteira1['TEMPO_ESPERA_DESPACHO_DIAS'] = (base_inteira1['Dt DDB'] - base_inteira1['Dt DIB']).dt.days
C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\1260596745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_inteira1['TEMPO_BENEF_DIAS'] = (base_inteira1['Dt DCB'] - base_inteira1['Dt DIB']).dt.days


In [11]:
# Criando uma função para agrupar os CIDs em categaorias
def classificar_cid(codigo):
    codigo = str(codigo)
    if codigo.startswith('F'):
        return 'Transtornos mentais e comportamentais'
    elif codigo.startswith('M'):
        return 'Doenças osteomusculares'
    elif codigo.startswith('I'):
        return 'Circulatório'
    elif codigo.startswith('A') or codigo.startswith('B'):
        return 'Doenças infecciosas e parasitárias'
    elif codigo.startswith('C') or codigo.startswith('D'):
        return 'Neoplasias (tumores)'
    elif codigo.startswith('E'):
        return 'Doenças endócrinas, nutricionais e metabólicas'
    elif codigo.startswith('G'):
        return 'Doenças do sistema nervoso'
    elif codigo.startswith('S') or codigo.startswith('T'):
        return 'Traumatismos e envenenamentos'
    elif codigo.startswith('O'):
        return 'Gravidez, parto e puerpério'
    else:
        return 'Outros'

In [12]:
# Aplicando a função de classificação de CID na coluna 'CID_NUM_NOME'
base_inteira1['CID_TIPO'] = base_inteira1['CID_NUM_NOME'].apply(classificar_cid)

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\1945921615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_inteira1['CID_TIPO'] = base_inteira1['CID_NUM_NOME'].apply(classificar_cid)


In [13]:
# Renomeando as colunas da base para facilitar a leitura
base_inteira1.rename(columns = {'Competência concessão': 'MES', 'Espécie_NOME': 'BENEF_TIPO', 'Despacho_NOME': 'CONCES_TIPO','Dt Nascimento': 'DT_NASCIMENTO','Dt DCB': 'DT_CESSACAO','Dt DDB': 'DT_DESPACHO','Dt DIB': 'DT_INICIO','Sexo.': 'SEXO','Clientela': 'ZONA','Ramo Atividade': 'RAMO','Idade': 'IDADE','Forma Filiação': 'STATUS_BENEFICIARIO','Classificador PA': 'PENSAO_ALIM', 'Vínculo dependentes': 'VINCULO_DEPEND', 'Qt SM RMI': 'QTD_SALARIO_MIN'}, inplace = True)

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\702939730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_inteira1.rename(columns = {'Competência concessão': 'MES', 'Espécie_NOME': 'BENEF_TIPO', 'Despacho_NOME': 'CONCES_TIPO','Dt Nascimento': 'DT_NASCIMENTO','Dt DCB': 'DT_CESSACAO','Dt DDB': 'DT_DESPACHO','Dt DIB': 'DT_INICIO','Sexo.': 'SEXO','Clientela': 'ZONA','Ramo Atividade': 'RAMO','Idade': 'IDADE','Forma Filiação': 'STATUS_BENEFICIARIO','Classificador PA': 'PENSAO_ALIM', 'Vínculo dependentes': 'VINCULO_DEPEND', 'Qt SM RMI': 'QTD_SALARIO_MIN'}, inplace = True)


In [14]:
# Selecionando as colunas de interesse para a base final
base_completa1 = base_inteira1[['MES','UF','ZONA','BENEF_TIPO','VINCULO_DEPEND','QTD_SALARIO_MIN','CONCES_TIPO','CID_TIPO','DT_INICIO','DT_DESPACHO','DT_CESSACAO','PENSAO_ALIM','TEMPO_BENEF_DIAS','TEMPO_ESPERA_DESPACHO_DIAS','RAMO','SEXO','IDADE','STATUS_BENEFICIARIO']]

In [15]:
# Criando uma nova coluna indicando se o tempo de espera do despacho é maior que o tempo do benefício
base_completa1['TEMPO_ESPERA_MAIOR_BENEF'] = (base_completa1['TEMPO_ESPERA_DESPACHO_DIAS'] >= base_completa1['TEMPO_BENEF_DIAS']).astype(int)

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\392798760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_completa1['TEMPO_ESPERA_MAIOR_BENEF'] = (base_completa1['TEMPO_ESPERA_DESPACHO_DIAS'] >= base_completa1['TEMPO_BENEF_DIAS']).astype(int)


In [16]:
# Criando uma função para agrupar o tempo de benefício em categorias
def categorizar_tempo(dias):
  if dias <= 31:
    return 'Até 1 mês'
  elif dias <= 90:
    return 'Acima de 1 mês a 3 meses'
  elif dias <= 180:
    return 'Acima de 3 meses a 6 meses'
  elif dias <= 365:
    return 'Acima de 6 meses a 1 ano'
  else:
    return 'Mais de 1 ano'

In [17]:
# Aplicando a função de categorização de tempo na coluna 'TEMPO_BENEF_DIAS'
base_completa1['TEMPO_FAIXA_BENEF'] = base_completa1['TEMPO_BENEF_DIAS'].apply(categorizar_tempo)

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\1786020827.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_completa1['TEMPO_FAIXA_BENEF'] = base_completa1['TEMPO_BENEF_DIAS'].apply(categorizar_tempo)


In [18]:
# Criando uma função para agrupar a quantidade de salário mínimo em categorias
def categorizar_salario(salario):
  if salario <= 1:
    return 'Até 1 SM'
  elif salario <= 3:
    return 'Entre 1 e 3 SM'
  elif salario <= 5:
    return 'Entre 3 e 5 SM'
  elif salario <= 10:
    return 'Entre 5 e 10 SM'
  else:
    return 'Mais de 10 SM'

In [19]:
# Aplicando a função de categorização por salário mínimo na coluna 'QTD_SALARIO_MIN'
base_completa1['QTD_SAL_MIN_FAIXA'] = base_completa1['QTD_SALARIO_MIN'].apply(categorizar_salario)

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\3598149710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_completa1['QTD_SAL_MIN_FAIXA'] = base_completa1['QTD_SALARIO_MIN'].apply(categorizar_salario)


In [20]:
# Categorizando a idade dos beneficiários em faixas etárias
bins_idade = [0, 20, 30, 40, 50, 60, 120]
labels_idade = ['0-20', '21-30', '31-40', '41-50', '51-60', '61+']
base_completa1['IDADE_FAIXA'] = pd.cut(base_completa1['IDADE'], bins=bins_idade, labels=labels_idade, right=True, include_lowest=True)

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\3342605966.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_completa1['IDADE_FAIXA'] = pd.cut(base_completa1['IDADE'], bins=bins_idade, labels=labels_idade, right=True, include_lowest=True)


In [21]:
# Criando uma função para obter a região do estado
def obter_regiao(estado):
    estado_str = str(estado)
    estado_normalizado = estado_str.strip().lower()
    mapa_estados_regioes = {
        # Norte
        'acre': 'Norte', 'ac': 'Norte',
        'amapa': 'Norte', 'amapá': 'Norte', 'ap': 'Norte',
        'amazonas': 'Norte', 'am': 'Norte',
        'para': 'Norte', 'pará': 'Norte', 'pa': 'Norte',
        'rondonia': 'Norte', 'rondônia': 'Norte', 'ro': 'Norte',
        'roraima': 'Norte', 'rr': 'Norte',
        'tocantins': 'Norte', 'to': 'Norte',
        # Nordeste
        'alagoas': 'Nordeste', 'al': 'Nordeste',
        'bahia': 'Nordeste', 'ba': 'Nordeste',
        'ceara': 'Nordeste', 'ceará': 'Nordeste', 'ce': 'Nordeste',
        'maranhao': 'Nordeste', 'maranhão': 'Nordeste', 'ma': 'Nordeste',
        'paraiba': 'Nordeste', 'paraíba': 'Nordeste', 'pb': 'Nordeste',
        'pernambuco': 'Nordeste', 'pe': 'Nordeste',
        'piaui': 'Nordeste', 'piauí': 'Nordeste', 'pi': 'Nordeste',
        'rio grande do norte': 'Nordeste', 'rn': 'Nordeste',
        'sergipe': 'Nordeste', 'se': 'Nordeste',
        # Centro-Oeste
        'goias': 'Centro-Oeste', 'goiás': 'Centro-Oeste', 'go': 'Centro-Oeste',
        'mato grosso': 'Centro-Oeste', 'mt': 'Centro-Oeste',
        'mato grosso do sul': 'Centro-Oeste', 'ms': 'Centro-Oeste',
        'distrito federal': 'Centro-Oeste', 'df': 'Centro-Oeste',
        # Sudeste
        'espirito santo': 'Sudeste', 'espírito santo': 'Sudeste', 'es': 'Sudeste',
        'minas gerais': 'Sudeste', 'mg': 'Sudeste',
        'rio de janeiro': 'Sudeste', 'rj': 'Sudeste',
        'sao paulo': 'Sudeste', 'são paulo': 'Sudeste', 'sp': 'Sudeste',
        # Sul
        'parana': 'Sul', 'paraná': 'Sul', 'pr': 'Sul',
        'rio grande do sul': 'Sul', 'rs': 'Sul',
        'santa catarina': 'Sul', 'sc': 'Sul',
    }

    return mapa_estados_regioes.get(estado_normalizado, 'Região não encontrada')

In [22]:
# Retirando a linha com preenchimento errado das colunas
base_completa2 = base_completa1[(base_completa1["UF"] != '81')]

In [23]:
# Aplicando afunção para obter a região do estado na coluna 'REGIAO_PAIS'
base_completa2['REGIAO_PAIS'] = base_completa2['UF'].apply(obter_regiao)

In [24]:
# Criando uma função para obter a população por região (ref: jul/2024)
def categorizar_populacao(regiao):
  if regiao == 'Norte':
    return 18669345
  elif regiao == 'Nordeste':
    return 57112096
  elif regiao == 'Centro-Oeste':
    return 17071595
  elif regiao == 'Sudeste':
    return 88617693
  elif regiao == 'Sul':
    return 31113021
  else:
    return '0'

In [32]:
# Agrupando a base através das colunas categorizadas
base_tratada = base_completa2.groupby(['MES','REGIAO_PAIS','CID_TIPO','SEXO','BENEF_TIPO','STATUS_BENEFICIARIO','IDADE_FAIXA'])['UF'].count().reset_index(name='QTD_BENEFICIOS')

C:\Users\beckz\AppData\Local\Temp\ipykernel_9336\4058857393.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  base_tratada = base_completa2.groupby(['MES','REGIAO_PAIS','CID_TIPO','SEXO','BENEF_TIPO','STATUS_BENEFICIARIO','IDADE_FAIXA'])['UF'].count().reset_index(name='QTD_BENEFICIOS')


In [33]:
# Aplicando afunção para obter a população por região na coluna 'POPULACAO'
base_tratada['POPULACAO'] = base_tratada['REGIAO_PAIS'].apply(categorizar_populacao)

In [34]:
# Normalizando os dados através da taxa por habitantes
base_tratada['TAXA_BENEFICIOS'] = (base_tratada['QTD_BENEFICIOS'] / base_tratada['POPULACAO']) * 100000

### Salvando a Base Final

In [35]:
# Salvando a base tratada em um arquivo CSV
base_tratada.to_csv('output/base_final.csv', index=False)